# Fuzzy C-Means (FCM)

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

## Geração de dados

In [2]:
def gerar_configuracao(mu_list, sigma_list, tamanhos, config_id):
    dfs = []
    for i, (mu, sigma2, n) in enumerate(zip(mu_list, sigma_list, tamanhos)):
        Sigma = np.diag(sigma2)
        data = np.random.multivariate_normal(mu, Sigma, n)
        df = pd.DataFrame(data, columns=["x1", "x2"])
        df["class"] = i + 1
        dfs.append(df)
    df_config = pd.concat(dfs, ignore_index=True)
    df_config["config"] = config_id
    return df_config

np.random.seed(42)  # reprodutibilidade

# -------------------------------
# Configuração 1
mu_1 = [[5, 0], [15, 5], [18, 14]]
sigma2_1 = [[81, 9], [9, 100], [25, 36]]
n1 = [200, 100, 50]
df1 = gerar_configuracao(mu_1, sigma2_1, n1, config_id=1)
l1 = "Classes elípticas de tamanhos diferentes"

# -------------------------------
# Configuração 2
mu_2 = [[0, 0], [30, 0], [12, 25]]
sigma2_2 = [[100, 100], [49, 49], [16, 16]]
n2 = [200, 100, 50]
df2 = gerar_configuracao(mu_2, sigma2_2, n2, config_id=2)
l2 = "Classes esféricas de tamanhos diferentes"

# -------------------------------
# Configuração 3
mu_3 = [[0, 0], [15, 5], [15, -5]]
sigma2_3 = [[100, 4], [100, 4], [100, 4]]
n3 = [100, 100, 100]
df3 = gerar_configuracao(mu_3, sigma2_3, n3, config_id=3)
l3 = "Classes elípticas de tamanhos iguais"

# -------------------------------
# Configuração 4
mu_4 = [[0, 0], [15, 0], [-15, 0]]
sigma2_4 = [[16, 16], [16, 16], [16, 16]]
n4 = [100, 100, 100]
df4 = gerar_configuracao(mu_4, sigma2_4, n4, config_id=4)
l4 = "Classes elípticas de tamanhos iguais"

# -------------------------------
# Configuração 5
mu_5 = [[5, 0], [15, 5], [10, -7], [3, 15]]
sigma2_5 = [[81, 9], [9, 100], [49, 16], [25, 25]]
n5 = [50, 50, 50, 50]
df5 = gerar_configuracao(mu_5, sigma2_5, n5, config_id=5)
l5 = "3 classes elípticas e 1 esférica"

# -------------------------------
# Configuração 6
mu_6 = [[5, 0], [15, 5], [12, -12], [7, 17]]
sigma2_6 = [[81, 9], [9, 100], [16, 16], [25, 25]]
n6 = [50, 50, 50, 50]
df6 = gerar_configuracao(mu_6, sigma2_6, n6, config_id=6)
l6 = "2 classes elípticas e 2 esféricas"

# -------------------------------
# Configuração 7
mu_7 = [[0, 0], [18, 0], [-18, 0], [0, -12]]
sigma2_7 = [[12, 12], [20, 20], [16, 16], [81, 20]]
n7 = [50, 50, 50, 50]
df7 = gerar_configuracao(mu_7, sigma2_7, n7, config_id=7)
l7 = "1 classe elíptica e 3 esféricas"

### Verificando as classes

In [3]:
def crisp_to_fuzzy(y, n_clusters): # transforma o dataset em fuzzy
    fuzzy_labels = np.zeros((len(y), n_clusters)) # cria uma array do dataset preenchida só com zeros
    for i, label in enumerate(y):
        fuzzy_labels[i, label] = 1 # com base na classe, o zero é substituído por um
    return fuzzy_labels

def inicializao_matriz_pertinencia(num_amostras, num_clusters):
    matriz_pertinencia = np.random.rand(num_amostras, num_clusters) # gera uma matriz inicial aleatória com valores entre 0 e 1
    matriz_pertinencia = matriz_pertinencia / matriz_pertinencia.sum(axis=1, keepdims=True) # normalização da matriz pra garantir que a soma dos graus dê um
    return matriz_pertinencia

def atualizacao_centroides(dados, matriz_pertinencia, m):
    matriz_pertinencia_m = matriz_pertinencia ** m # preparação dos graus de pertinência
    centroides = np.dot(matriz_pertinencia_m.T, dados) / np.sum(matriz_pertinencia_m.T, axis=1, keepdims=True) # fórmula para o cálculo dos centroides
    return centroides

def atualizacao_matriz_pertinencia(dados, centroides, m):
    matriz_distancias = np.linalg.norm(dados[:, np.newaxis] - centroides, axis=2) ** 2
    matriz_distancias = np.fmax(matriz_distancias, np.finfo(np.float64).eps) # evita que matriz_distancias seja 0, np.finfo... é o menor número maior que zero aaqui
    matriz_distancias_inversa = 1 / matriz_distancias
    potencia = 1 / (m-1)
    matriz_pertinencia_atualizada = matriz_distancias_inversa ** potencia / np.sum(matriz_distancias_inversa ** potencia, axis=1, keepdims=True) # fórmula para atualizar os graus de pertinência
    return matriz_pertinencia_atualizada

def fcm(dados, num_clusters, m=2, max_iter=1000, erro=1e-5):
    num_amostras = dados.shape[0]
    matriz_pertinencia = inicializao_matriz_pertinencia(num_amostras, num_clusters)
    for _ in range(max_iter): # primeiro critério de parada
        centroides = atualizacao_centroides(dados, matriz_pertinencia, m)
        nova_matriz_pertinencia = atualizacao_matriz_pertinencia(dados, centroides, m)
        if np.linalg.norm(nova_matriz_pertinencia - matriz_pertinencia) < erro: # segundo critério de parada
            break
        matriz_pertinencia = nova_matriz_pertinencia
    return centroides, matriz_pertinencia

def pertinence_distance(delta_k, delta_k_linha, c):
    # calcula a distância entre δ_k e δ_k' (matrizes de pertinência)
    return (1/c) * np.sum((delta_k - delta_k_linha) ** 2)

def fuzzy_rand_index(particao1, particao2, c):
    n = particao1.shape[0]
    total_sum = 0

    for k in range(n):
        for k_linha in range(k+1, n):
            if k != k_linha:
                # calcula a métrica para P
                delta_k = particao1[k]
                delta_k_prime = particao1[k_linha]
                EP = 1 - pertinence_distance(delta_k, delta_k_prime, c)

                # calcula a métrica para Q
                delta_k_Q = particao2[k]
                delta_k_prime_Q = particao2[k_linha]
                EQ = 1 - pertinence_distance(delta_k_Q, delta_k_prime_Q, c)

                total_sum += np.abs(EP - EQ) # soma a diferença absoluta entre EP e EQ

    denominador = n * (n - 1) / 2
    if denominador == 0:
        raise ValueError

    return 1- (total_sum / denominador)

def simulacao_monte_carlo(dados, part_dif, num_clusters, num_trials):
    indices_rand = []
    for _ in range(num_trials):
        #print(_)
        centroides, matriz_pertinencia = fcm(dados, num_clusters)
        predicted_labels = np.argmax(matriz_pertinencia, axis=1)
        idx_rand = fuzzy_rand_index(part_dif, predicted_labels, num_clusters)
        indices_rand.append(idx_rand)
    mean_rand_index = np.mean(indices_rand)
    std_rand_index = np.std(indices_rand)
    return mean_rand_index, std_rand_index
i = 1
for df in [df1, df2, df3, df4, df5, df6, df7]:
    if i == 5 or i == 6 or i == 7:
        num_clusters = 4
    else:
        num_clusters = 3
    df.drop("config", axis=1, inplace=True)
    if i == 5 or i == 6 or i == 7:
        df["class"].replace({1: 0, 2: 1, 3: 2, 4: 3}, inplace=True)
    else: 
        df["class"].replace({1: 0, 2: 1, 3: 2}, inplace=True)
    part_dif = df["class"].values
    df.drop("class", axis=1, inplace=True)
    dados = df.to_numpy()
    part_dif = crisp_to_fuzzy(part_dif, num_clusters)
    num_trials = 100
    media_indice_rand, dp_indice_rand = simulacao_monte_carlo(dados, part_dif, num_clusters, num_trials)
    print(f"Monte Carlo FCM Clustering Results for Config {i}")
    print(f"Mean Rand Index: {media_indice_rand:.4f}") # 4 casas decimais
    print(f"Standard Deviation of Rand Index: {dp_indice_rand:.4f}") # 4 casas decimais
    print("\n")
    i += 1

Monte Carlo FCM Clustering Results for Config 1
Mean Rand Index: 0.5801
Standard Deviation of Rand Index: 0.0598


Monte Carlo FCM Clustering Results for Config 2
Mean Rand Index: 0.6487
Standard Deviation of Rand Index: 0.0294


Monte Carlo FCM Clustering Results for Config 3
Mean Rand Index: 0.5320
Standard Deviation of Rand Index: 0.0342


Monte Carlo FCM Clustering Results for Config 4
Mean Rand Index: 0.6725
Standard Deviation of Rand Index: 0.0065


Monte Carlo FCM Clustering Results for Config 5
Mean Rand Index: 0.4863
Standard Deviation of Rand Index: 0.0301


Monte Carlo FCM Clustering Results for Config 6
Mean Rand Index: 0.4937
Standard Deviation of Rand Index: 0.0490


Monte Carlo FCM Clustering Results for Config 7
Mean Rand Index: 0.5290
Standard Deviation of Rand Index: 0.0258




In [4]:
# Parâmetros das configurações
params_config_12 = [
    {'mu': [-16, -5], 'sigma': [20, 20], 'n': 50},
    {'mu': [-8, 8], 'sigma': [13, 13], 'n': 100},
    {'mu': [0, 0], 'sigma': [6, 6], 'n': 200},
]

params_config_13 = [
    {'mu': [7, -6], 'sigma': [50, 5], 'n': 100},
    {'mu': [0, 0], 'sigma': [2, 50], 'n': 100},
    {'mu': [12, 0], 'sigma': [50, 5], 'n': 100},
]

# Faixa para ruído e semente aleatória
noise_range = [-100, 50]
np.random.seed(42)

# Função de geração de dados com ruído
def generate_data(config_params, noise_percent):
    data_all = []
    for class_idx, param in enumerate(config_params, start=1):
        mu = np.array(param['mu'])
        sigma_diag = np.diag(param['sigma'])
        n = param['n']
        n_noise = int(n * noise_percent / 100)
        n_signal = n - n_noise

        real_data = np.random.multivariate_normal(mu, sigma_diag, n_signal)
        labels_real = np.full((n_signal,), class_idx)

        noise_data = np.random.uniform(noise_range[0], noise_range[1], size=(n_noise, 2))
        labels_noise = np.full((n_noise,), 0)

        data = np.vstack([real_data, noise_data])
        labels = np.concatenate([labels_real, labels_noise])

        df = pd.DataFrame(data, columns=['x1', 'x2'])
        df['class'] = labels
        data_all.append(df)

    return pd.concat(data_all, ignore_index=True)

# Configurações e títulos
configs = [
    (params_config_12, 10),
    (params_config_12, 20),
    (params_config_12, 30),
    (params_config_13, 10),
    (params_config_13, 20),
    (params_config_13, 30),
]

titles = [
    "Configuração 12 - 10% ruído",
    "Configuração 12 - 20% ruído",
    "Configuração 12 - 30% ruído",
    "Configuração 13 - 10% ruído",
    "Configuração 13 - 20% ruído",
    "Configuração 13 - 30% ruído",
]

# Geração dos DataFrames separadamente
dfs_por_config = {}

for (params, noise), title in zip(configs, titles):
    df = generate_data(params, noise)
    df['classe_legenda'] = df['class'].replace(0, 'ruído')
    dfs_por_config[title] = df

In [5]:
for nome_config, df in dfs_por_config.items():
    num_clusters = 4
    labels = df["class"].values
    labels = crisp_to_fuzzy(labels, num_clusters)
    df.drop("class", axis=1, inplace=True)
    df.drop("classe_legenda", axis=1, inplace=True)
    dados = df.to_numpy()
    num_trials = 100
    media_indice_rand, dp_indice_rand = simulacao_monte_carlo(dados, labels, num_clusters, num_trials)
    print("\n")
    print(f"Resultados de Monte Carlo para {nome_config} ({num_trials} tentativas)")
    print(f"Média do Índice Rand: {media_indice_rand:.4f}")
    print(f"Desvio Padrão do Índice Rand: {dp_indice_rand:.4f}")



Resultados de Monte Carlo para Configuração 12 - 10% ruído (100 tentativas)
Média do Índice Rand: 0.6260
Desvio Padrão do Índice Rand: 0.1399


Resultados de Monte Carlo para Configuração 12 - 20% ruído (100 tentativas)
Média do Índice Rand: 0.5962
Desvio Padrão do Índice Rand: 0.1199


Resultados de Monte Carlo para Configuração 12 - 30% ruído (100 tentativas)
Média do Índice Rand: 0.5412
Desvio Padrão do Índice Rand: 0.1457


Resultados de Monte Carlo para Configuração 13 - 10% ruído (100 tentativas)
Média do Índice Rand: 0.4884
Desvio Padrão do Índice Rand: 0.1531


Resultados de Monte Carlo para Configuração 13 - 20% ruído (100 tentativas)
Média do Índice Rand: 0.4826
Desvio Padrão do Índice Rand: 0.1934


Resultados de Monte Carlo para Configuração 13 - 30% ruído (100 tentativas)
Média do Índice Rand: 0.5232
Desvio Padrão do Índice Rand: 0.1311
